In [1]:
import os.path
import re

from common.consts import PATTERN_FILE_FORMAT__MSI, PATTERN_TYPE__BROADCAST, PATTERN_TYPE__BEAMFORMING_ELEMENT, PATTERN_TYPE__BEAMSWITCHING_SERVICE
from common.beamforming_antenna_generator import BeamformingAntennaGenerator
from common.re_filter import ReFilter
from common.pattern_name_param_extractor import PatternNameParamExtractor
from common.pattern_payload_param_extractor import PatternPayloadParamExtractor
from common.pattern_name_param_selector import PatternNameParamSelector

In [2]:
generator = BeamformingAntennaGenerator({
    # General parameters
    # 'src_folder': 'D:\\Cell Planning\\Antenna model processing\\Library\\mMIMO\\mMIMO Antennas\\Nokia\\AQQA',
    'src_folder': 'D:\\TP\\mMIMO Antennas\\Nokia\\AQQA',
    'pattern_file_format': PATTERN_FILE_FORMAT__MSI,
    'version': '7.4',
    'filename': 'AQQA_32T32R.pafx',
    'name': 'AQQA MIMO 32T32R 96 ELEM',
    'type': 'Cellular',
    'comment': 'AQQA MIMO 32T32R 96 ELEM',
    'manufacturer': 'Nokia',
    'cost': 0,
    'cost_unit': 'USD',
    'length_cm': 61,
    'width_cm': 38,
    'depth_cm': 10,
    'weight_kg': 27,
    'wind_load_factor': 587,
    'supp_elec_tilt': True,
    'supp_elec_azimuth': False,
    'supp_elec_beamwidth': False,
    'cont_adj_elec_tilt': False,
    
    # Source file filter
    'src_file_re_filter': ReFilter(
        allow=[
            '.*BeamPattern.*Optimized.*(Envelope|RefBeam).*\.msi$',
        ],
        deny=[
            '.*TypeApproval.*',
            '.*3GPP.*',
            '.*PatternEnvelope.*',
        ],
    ),
    
    # Parameter extractors
    'pattern_name_extractor': PatternNameParamExtractor(
        path_part='basename',
        extract_re='(?P<cg>.+)\..{3}$',
    ),
    'scenario_extractor': PatternNameParamExtractor(
        extract_re='.*-(?P<cg>\d+deg.+)-(Envelope|RefBeam).*',
        post_capture_proc=lambda r: re.sub('-p\d+-ap\d+-', '__', r),
    ),
    'v_port_name_extractor': PatternNameParamExtractor(
        extract_re='.*-(?P<cg>\d+deg.+)-(Envelope|RefBeam).*',
        post_capture_proc=lambda r: re.sub('-p\d+-ap\d+-', '__', r),
    ),
    'pattern_type_extractor': PatternNameParamExtractor(
        path_part='basename',
        pre_capture_proc=lambda r: r.lower(),
        post_capture_proc=lambda r: PATTERN_TYPE__BROADCAST if 'ssb' in r else PATTERN_TYPE__BEAMSWITCHING_SERVICE,
    ),
    'center_freq_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: int(float(payload.header['FREQUENCY']))
    ),
    'min_freq_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: int(float(payload.header['FREQUENCY']) - 100)
    ),
    'max_freq_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: int(float(payload.header['FREQUENCY']) + 100)
    ),
    'electrical_tilt_extractor': PatternNameParamExtractor(
        path_part='basename',
        pre_capture_proc=lambda r: r.lower(),
        extract_re='.*-ap(?P<cg>\d+)-.*',
        post_capture_proc=lambda r: int(r),
    ),
    'polarization_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 'Vertical',
    ),
    'polarization_type_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: None,
    ),
    'v_port_number_of_ports_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 4,
    ),
    'horiz_number_of_elements_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 4,
    ),
    'horiz_sep_dist_cm_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 4.3,
    ),
    'vert_number_of_elements_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 4,
    ),
    'vert_sep_dist_cm_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 17.7,
    ),
    'beamswitching_service_name_extractor': PatternNameParamExtractor(
        post_capture_proc=lambda r: 'PDSCH',
    ),
    'beamswitching_beam_id_extractor': PatternNameParamExtractor(
        path_part='basename',
        extract_re='.*(Envelope|RefBeam-)(?P<cg>.+)\.msi',
    ),
    'beamswitching_horiz_angle_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: payload.horiz_pap_pattern.get_boresight_deg(),
    ),
    'beamswitching_vert_angle_extractor': PatternPayloadParamExtractor(
        extract_fn=lambda payload: payload.vert_pap_pattern.get_boresight_deg(),
    ),
    
    # Parameter selectors
    'scenario_selector': None,
    'v_port_name_selector': None,
})

In [3]:
extracted_tags = generator.list_extracted_tags(detailed=False)


Extracted tags list

[scenario]:
   1.  120degAzOp__#1..........6 items
   2.  120degAzOp__#2..........10 items
   3.  120degAzOp__#2#2........18 items
   4.  120degAzOp__#3#1........18 items
   5.  120degAzOp__#3#3........26 items
   6.  120degAzOp__#3#3#2......34 items
   7.  120degAzOp__#3#3#2b.....34 items
   8.  120degAzOp__#4..........18 items
   9.  120degAzOp__#4#2........26 items
  10.  120degAzOp__#4#4........34 items
  11.  120degAzOp__#5#1........26 items
  12.  120degAzOp__#5#3........34 items
  13.  120degAzOp__#6..........26 items
  14.  120degAzOp__#6#2........34 items
  15.  120degAzOp__#8..........34 items
  16.  40degAzOp__#2#2#2#2.....34 items
  17.  90degAzOp__#1...........6 items
  18.  90degAzOp__#2...........10 items
  19.  90degAzOp__#2#2.........18 items
  20.  90degAzOp__#2#2#2#2.....34 items
  21.  90degAzOp__#3#1.........18 items
  22.  90degAzOp__#3#3.........26 items
  23.  90degAzOp__#3#3#2.......34 items
  24.  90degAzOp__#3#3#2b......34 items
  25.  9

In [11]:
generator.generate(os.path.abspath('.\output'))


Scenario > Virutal port > Virutal band > Pattern assignments
{
  "120degAzOp__#1": {
    "uid": "2",
    "name": "120degAzOp__#1",
    "horiz_number_of_elements": 4,
    "horiz_sep_dist_cm": 4.3,
    "vert_number_of_elements": 4,
    "vert_sep_dist_cm": 17.7,
    "v_ports": {
      "120degAzOp__#1": {
        "uid": "3",
        "name": "120degAzOp__#1",
        "number_of_ports": 4,
        "polarization": "Vertical",
        "polarization_type": null,
        "v_bands": {
          "3400-3600": {
            "min_freq": 3400,
            "max_freq": 3600,
            "supp_elec_tilt": true,
            "supp_elec_azimuth": false,
            "supp_elec_beamwidth": false,
            "cont_adj_elec_tilt": false,
            "broadcast_patterns": [
              "AQQA-Optimized-120degAzOp-p00-ap07-#1-Envelope_L1-SSB"
            ],
            "electrical_controller_name": "Controller 1",
            "use_elec_params_for_bs_service_patterns": false,
            "beamforming_element_pa